In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Install dependencies

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pylab as plt
import cv2
import os

#import data
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import cv2
import os

# Read in data files
BASE_DIR = '../input/nfl-health-and-safety-helmet-assignment'
path = '../input/nfl-health-and-safety-helmet-assignment/train'

# Labels and sample submission
labels = pd.read_csv(f'{BASE_DIR}/train_labels.csv')
ss = pd.read_csv(f'{BASE_DIR}/sample_submission.csv')

# Player tracking data
tr_tracking = pd.read_csv(f'{BASE_DIR}/train_player_tracking.csv')
te_tracking = pd.read_csv(f'{BASE_DIR}/test_player_tracking.csv')

# Baseline helmet detection labels
tr_helmets = pd.read_csv(f'{BASE_DIR}/train_baseline_helmets.csv')
te_helmets = pd.read_csv(f'{BASE_DIR}/test_baseline_helmets.csv')

# Extra image labels
img_labels = pd.read_csv(f'{BASE_DIR}/image_labels.csv')

In [ ]:
# !pip install pyyaml==5.1
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
# !pip install pyyaml==5.1
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html
    
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
assert torch.__version__.startswith("1.9")   # please manually install torch 1.9 if Colab changes its default version

# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
# from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog   


# **prepare data**

In [ ]:
df_train_player_tracking = pd.read_csv(f'{BASE_DIR}/train_player_tracking.csv')
df = pd.read_csv("../input/nfl-health-and-safety-helmet-assignment/train_labels.csv")

files = df.video.unique().tolist()
df_files = pd.DataFrame(files)
df_files  = df_files.rename(columns={0:"video_frame"})
df_files['gameKey']  = df_files["video_frame"].str.split("_").str[-3].astype('int64')
df_files['playID'] = df_files["video_frame"].str.split("_").str[-2].astype('int64')
df_files['View1']  = df_files["video_frame"].str.split("_").str[-1]
df_files['View']  = df_files["View1"].str.split(".").str[-2]

# videos from sideline

df_sideline_videos = df_files.query('View == "Sideline"')
df_sideline_videos = df_sideline_videos.reset_index(drop=True)

# videos from endzone

df_endzone_videos = df_files.query('View == "Endzone"')
df_endzone_videos = df_endzone_videos.reset_index(drop=True)


# unused here
"""
labels = pd.read_csv(f'{BASE_DIR}/train_labels.csv')
labels["team"] = labels["label"].str[0].replace("H", "Home").replace("V", "Away")
labels['x'] = labels['left'] + labels['width']/2
labels['y'] = labels['top'] + labels['height']/2
labels['est_frame'] = labels['frame']
"""


# helmets bounding box data

sk_helmets = pd.read_csv(f'{BASE_DIR}/train_baseline_helmets.csv')
sk_helmets['gameKey'] = sk_helmets["video_frame"].str.split("_").str[-4].astype('int64')
sk_helmets['playID'] = sk_helmets["video_frame"].str.split("_").str[-3].astype('int64')
sk_helmets['est_frame']  = sk_helmets["video_frame"].str.split("_").str[-1].astype('int64')
sk_helmets['View']  = sk_helmets["video_frame"].str.split("_").str[-2]
sk_helmets['x'] = sk_helmets['left'] + sk_helmets['width']/2
sk_helmets['y'] = (sk_helmets['top'] + sk_helmets['height']/2)
sk_helmets = sk_helmets.query('conf > 0.4 ')
tr_baseline_helmets = sk_helmets.copy()
tr_baseline_helmets


# **Load image**

In [ ]:
k = 20
frame = 4

source = os.path.join(path, df_endzone_videos['video_frame'][k])    
cap = cv2.VideoCapture(source)

for i in range(frame):
    ret,img = cap.read()

plt.imshow(img)

# **Predict keypoints**

In [ ]:
# Inference with a keypoint detection model

cfg = get_cfg()   # get a fresh new config

# cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")) # (a more lightweight network)
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_X_101_32x8d_FPN_3x.yaml"))

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model

# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")  # (a more lightweight network)
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_X_101_32x8d_FPN_3x.yaml")

predictor = DefaultPredictor(cfg)
outputs = predictor(img)
v = Visualizer(img[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

plt.figure(figsize=(20,20))
plt.imshow(out.get_image()[:, :, ::-1])

In [ ]:
img2 = img.copy()

# --- Load keypoints ---

locs = outputs['instances'].get('pred_keypoints').detach().cpu().numpy().astype(np.int32)
scores = outputs['instances'].get('pred_keypoints').detach().cpu().numpy()[:,:,2]

# --- Load helmet bounding boxes --- (and convert them to a numpy array)

video_name = df_endzone_videos["gameKey"][k]
play_name = df_endzone_videos['playID'][k]
helmets  = sk_helmets.query('gameKey == @video_name & est_frame == 4 & playID == @play_name & View == "Endzone"')

# helmet bounding boxes (Number of helmets, x0, y0, x1, y1)
coords = np.array([helmets.left.values, helmets.top.values, helmets.left.values + helmets.width.values, helmets.top.values + helmets.height.values])

# --- match helmets with keypoints ---

limb_helmet_match = np.zeros(coords.shape[1]).astype(np.int8) - 1
limb_helmet_confidence = np.zeros(coords.shape[1])
limb_helmet_deltas = np.zeros([coords.shape[1],2]) # distance between the head and the legs in the form of (Number of helmets, (dx, dy)).

# a function to check whether a given keypoint is inside a given bounding box
is_inside = lambda head_keycoord, coords: (coords[0] < head_keycoord[0] < coords[2]) and (coords[1] < head_keycoord[1] < coords[3,]) 


for j in range(locs.shape[0]):
    for i in range(limb_helmet_match.shape[0]):
        
        # find how many of a face keypoints are inside a helmet bounding box
        
        eyeloc  = locs[j,0,:]
        eyeloc2  = locs[j,1,:]
        noseloc  = locs[j,2,:]
        
        head_keycoords = (eyeloc, eyeloc2, noseloc)
        keypoints_inside = sum([int(is_inside(head_keycoord, coords[:,i])) for head_keycoord in head_keycoords])
        
        # if there are more keypoints in the bounding box than a previous result, match them
        
        if limb_helmet_confidence[i] < keypoints_inside:
            limb_helmet_confidence[i] = keypoints_inside
            limb_helmet_match[i] = j

# ---

left_ankle = 15
right_ankle = 16
conf = 0.02 # confidence of model in prediction for a given keypoint

# ---

for i in range(limb_helmet_match.shape[0]):
    
    if limb_helmet_match[i] >= 0:
        
        limb_coord = limb_helmet_match[i]
        
        if scores[limb_coord,right_ankle] > conf < scores[limb_coord,left_ankle]:
            
            # get coordinates for the average legs pose
            legs_pose = (0.5*(locs[limb_coord,left_ankle,:2] + locs[limb_coord,right_ankle,:2])).round().astype(np.int32)
            
            # get the center of the helmet bounding box
            helmet_pose = np.array(
                [
                    (0.5*(coords[0,i]+coords[2,i])),
                    (0.5*(coords[1,i]+coords[3,i]))
                ]
            ).round().astype(np.int32)
        
        # calculate distances
        limb_helmet_deltas[i,:] = legs_pose-helmet_pose
        
        #  --- Draw lines from helmets to legs ---
        cv2.line(img2, legs_pose, helmet_pose, (255,0,255),2)


# --- Draw helmets ---

for i in range(coords.shape[1]):
    cv2.rectangle(img2,(coords[0,i],coords[1,i]),(coords[2,i],coords[3,i]),(255,0,0),1)

# --- Draw keypoints ---

for i in range(locs.shape[0]):
    loc  = locs[i,left_ankle,:]
    loc2  = locs[i,right_ankle,:]
    
    eyeloc  = locs[i,0,:]
    eyeloc2  = locs[i,1,:]
    noseloc  = locs[i,2,:]
    
    loc3 = (0.5*(loc2[:2]+loc[:2])).round().astype(np.int32)
    
    if scores[i,left_ankle] > conf:
        cv2.circle(img2, loc[:2], 4, (0,100,255), 2)
        #cv2.putText(img2, f'res: {format(scores[i,left_ankle],".2f")}', loc[:2], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)
    
    if scores[i,right_ankle] > conf:
        cv2.circle(img2, loc2[:2],4, (0,100,255), 2)    
        #cv2.putText(img2, f'res: {format(scores[i,right_ankle],".2f")}', loc2[:2], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)
    
    if scores[i,right_ankle] > conf < scores[i,left_ankle]:
        cv2.circle(img2,loc3,4,(0,255,100),6)
        
    cv2.circle(img2, eyeloc[:2], 1, (0,255,255), 4)
    cv2.circle(img2, eyeloc2[:2],1, (0,255,255), 4)
    cv2.circle(img2, noseloc[:2],1, (0,255,255), 4)

plt.figure(figsize=(20,20))
plt.imshow(img2)

In [ ]:
print("helmet keypoints match: ",limb_helmet_match)
print("helmet to legs offset (deltas): \n\n", limb_helmet_deltas)